## 3 Datasets:
- MNIST
- Spambase
- 20NG

For each, perform 2 classification algorithms:
- L2-reg Logistic Regression
- Decision Trees


allowed to use library for classification. 

allowed to use library to process data in appropriate formats.

You are required to explain/analyze the model trained in terms of features : for each of the 6 runs list the top F=30 features. 

For the Regression these correspond to the highest-absolute-value F coefficients; 

for Decision Tree they are the first F splits. In particular for Decision Tree on 20NG, report performance for two tree sizes ( by depths of the tree, or number of leaves, or number of splits )

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as torch
from sklearn.datasets import _twenty_newsgroups
from torchvision import datasets, transforms

# MNIST

In [ ]:
# Load the MNIST dataset
mnist_dataset = datasets.MNIST(root='data', download=True, transform=transforms.ToTensor())
features = mnist_dataset.data
labels = mnist_dataset.targets

# Normalize the features
features = features / 255.0

In [ ]:
# using pytorch to perform logistic regression:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))
    
# Initialize the model, loss function, and optimizer
input_dim = 28 * 28  # MNIST images are 28x28 pixels
output_dim = 10  # 10 classes for digits 0-9
model = LogisticRegression(input_dim, output_dim)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Prepare the data for training
features = features.view(-1, input_dim).float()
labels = labels.long()

# Training loop
num_epochs = 5
batch_size = 64
num_batches = len(features) // batch_size
for epoch in range(num_epochs):
    for i in range(num_batches):
        # Get the batch data
        batch_features = features[i * batch_size:(i + 1) * batch_size]
        batch_labels = labels[i * batch_size:(i + 1) * batch_size]
        
        # Forward pass
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(num_batches):
        batch_features = features[i * batch_size:(i + 1) * batch_size]
        batch_labels = labels[i * batch_size:(i + 1) * batch_size]
        
        outputs = model(batch_features)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    accuracy = correct / total
print(f'Accuracy of the model on the MNIST test images: {accuracy * 100:.2f}%')


Epoch [1/5], Loss: 2.0257
Epoch [2/5], Loss: 1.9207
Epoch [3/5], Loss: 1.8654
Epoch [4/5], Loss: 1.8299
Epoch [5/5], Loss: 1.8045
Accuracy of the model on the MNIST test images: 84.15%


In [5]:
# Analyze model in terms of features, d = top 30 features
def analyze_model(model, input_dim):
    weights = model.linear.weight.data.numpy()
    top_features = np.argsort(np.abs(weights), axis=1)[:, -30:]  # Get top 30 features for each class
    return top_features
top_features = analyze_model(model, input_dim)

print("Top 30 features for each class:")
for i in range(output_dim):
    print(f"Class {i}: {top_features[i]}")

Top 30 features for each class:
Class 0: [492 437 460 517 377 351 350 489 490 353 461 409 464 379 432 491 352 381
 433 378 405 406 463 434 436 462 407 435 380 408]
Class 1: [262 377 381 263 438 407 433 403 402 294 375 493 319 410 382 462 465 295
 379 437 409 489 461 323 378 351 322 434 350 406]
Class 2: [377 289 152 516 153 125 656 292 316 344 567 569 293 348 321 372 373 374
 630 375 657 658 290 320 345 346 317 347 318 319]
Class 3: [458 323 545 344 485 289 178 292 515 491 377 462 290 486 514 519 345 262
 463 291 461 318 518 544 317 516 488 490 489 517]
Class 4: [409 155 294 600 436 597 431 374 465 542 372 373 266 402 429 570 400 184
 430 209 464 212 238 239 183 182 401 428 210 211]
Class 5: [356 220 435 328 516 489 402 515 219 218 485 354 191 461 353 189 299 375
 348 488 355 320 376 326 346 374 325 298 327 347]
Class 6: [541 240 214 325 657 486 542 626 659 544 654 514 268 299 297 215 572 213
 515 656 296 543 242 298 655 271 269 243 241 270]
Class 7: [266 260 240 296 428 156 431 298 35

In [6]:
# Using decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the MNIST dataset
mnist_dataset_train = datasets.MNIST(root='data', train = True, download=True)

mnist_dataset = datasets.MNIST(root='data', train = False, download=True)
print(mnist_dataset_train.data.shape)
print(mnist_dataset.data.shape)
# Train features, labels
features = mnist_dataset_train.data
labels = mnist_dataset_train.targets

# Test features, labels
test_features = mnist_dataset.data
test_labels = mnist_dataset.targets

# convert to numpy arrays:
features = features.numpy()
labels = labels.numpy()

test_features = test_features.numpy()
test_labels = test_labels.numpy()

# Normalize the features
features = features / 255.0
test_features = test_features / 255.0

# Reshape the features to 2D array
features = features.reshape(features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the Decision Tree Classifier
dt_classifier.fit(features, labels)

# Predict on the testing set
dt_predictions = dt_classifier.predict(test_features)

# Calculate accuracy
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print(f'Accuracy of the Decision Tree Classifier on the MNIST dataset: {dt_accuracy * 100:.2f}%')



torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])
Accuracy of the Decision Tree Classifier on the MNIST dataset: 87.64%


# Spambase


In [7]:
spambase = pd.read_csv('spambase.data', header=None, sep=',')
spambase

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [8]:
# Split the dataset into features and labels
s_labels = spambase.iloc[:, -1]
s_features = spambase.iloc[:, :-1]


In [9]:
# Normalize the features
s_features = (s_features - s_features.mean()) / s_features.std()

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
s_features_train, s_features_test, s_labels_train, s_labels_test = train_test_split(s_features, s_labels, test_size=0.2, random_state=42)


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Using logistic regression

# Initialize the Logistic Regression model
logistic_model = LogisticRegression()

# Train the model
logistic_model.fit(s_features_train, s_labels_train)

# Predict on the testing set
logistic_predictions = logistic_model.predict(s_features_test)

# Calculate accuracy
logistic_accuracy = accuracy_score(s_labels_test, logistic_predictions)
print(f'Accuracy of the Logistic Regression model on the Spambase dataset: {logistic_accuracy * 100:.2f}%')

Accuracy of the Logistic Regression model on the Spambase dataset: 91.97%


In [11]:
# Decision tree classifier

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the Decision Tree Classifier
dt_classifier.fit(s_features_train, s_labels_train)

# Predict on the testing set
dt_predictions = dt_classifier.predict(s_features_test)

# Calculate accuracy
dt_accuracy = accuracy_score(s_labels_test, dt_predictions)
print(f'Accuracy of the Decision Tree Classifier on the Spambase dataset: {dt_accuracy * 100:.2f}%')


Accuracy of the Decision Tree Classifier on the Spambase dataset: 91.31%


# 20NG

In [12]:
ng_train = _twenty_newsgroups.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
ng_train_feat = ng_train.data
ng_train_labels = ng_train.target

ng_test = _twenty_newsgroups.fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
ng_test_feat = ng_test.data
ng_test_labels = ng_test.target

In [13]:
# TFIDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TFIDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit and transform the training features
ng_train_features = tfidf_vectorizer.fit_transform(ng_train_feat)

# Transform the testing features
ng_test_features = tfidf_vectorizer.transform(ng_test_feat)


In [14]:
# Using logistic regression

# Initialize the Logistic Regression model
logistic_model = LogisticRegression()

# Train the model
logistic_model.fit(ng_train_features, ng_train_labels)

# Predict on the testing set
logistic_predictions = logistic_model.predict(ng_test_features)

# Calculate accuracy
logistic_accuracy = accuracy_score(ng_test_labels, logistic_predictions)
print(f'Accuracy of the Logistic Regression model on the 20 Newsgroups dataset: {logistic_accuracy * 100:.2f}%')

Accuracy of the Logistic Regression model on the 20 Newsgroups dataset: 64.83%


## Q2: PCA Using library (sklearn compatible)

In [15]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from torchvision import datasets, transforms
from pca import pca

In [16]:
# Load the MNIST dataset
mnist_dataset = datasets.MNIST(root='data', download=True, transform=transforms.ToTensor())
features = mnist_dataset.data
labels = mnist_dataset.targets

# Normalize the features
features = features / 255.0

